# Snippet Sniper

### Welcome on a wild ride with the John Wick in the coding arena as it accepts your contracts 

Allows you to perform various tasks on given code snippets:

- Add comments
- Explain what the code does
- Writes comprehensive unit tests
- Fixes (potential) errors in the code

In [ ]:
import os
import logging
from enum import StrEnum
from getpass import getpass

import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv


load_dotenv(override=True)

In [ ]:
logging.basicConfig(level=logging.WARNING)

logger = logging.getLogger('sniper')
logger.setLevel(logging.DEBUG)

## Free Cloud Providers

Grab your free API Keys from these generous sites:

- https://ollama.com/

### Secrets Helpers

In [ ]:
def get_secret_in_google_colab(env_name: str) -> str:
    try:
      from google.colab import userdata
      return userdata.get(env_name)
    except Exception:
      return ''


def get_secret(env_name: str) -> str:
    '''Gets the value from the environment(s), otherwise ask the user for it if not set'''
    key = os.environ.get(env_name) or get_secret_in_google_colab(env_name)

    if not key:
        key = getpass(f'Enter {env_name}:').strip()

    if key:
        logger.info(f'✅ {env_name} provided')
    else:
        logger.warning(f'❌ {env_name} not provided')
    return key.strip()

### Set up model(s)

In [ ]:
class Provider(StrEnum):
    OLLAMA = 'Ollama'
    OPENROUTER = 'OpenRouter'

clients: dict[Provider, OpenAI] = {}

if api_key := get_secret('OLLAMA_API_KEY'):
    clients[Provider.OLLAMA] = OpenAI(api_key=api_key, base_url='https://ollama.com/v1')

model = 'qwen3-coder:480b-cloud'
client = clients.get(Provider.OLLAMA)
if not client:
    raise Exception('No client found')

## Tasks

In [ ]:
class Task(StrEnum):
  COMMENTS = 'Comments'
  UNIT_TESTS = 'Unit Tests'
  FIX_CODE = 'Fix Code'
  EXPLAIN = 'Explain'


def perform_tasks(tasks, code):
  logger.info(f'Performing tasks: {tasks}')

  steps = []
  if Task.COMMENTS in tasks:
    steps.append('Add documentation comments to the given code. If the method name and parameters are self-explanatory, skip those comments.')
  if Task.UNIT_TESTS in tasks:
    steps.append('Add a thorough unit tests considering all edge cases to the given code.')
  if Task.FIX_CODE in tasks:
    steps.append('You are to fix the given code, if it has any issues.')
  if Task.EXPLAIN in tasks:
    steps.append('Explain the given code.')

  system_prompt = f'''
  You are an experienced polyglot software engineer and given a code you can
  detect what programming language it is in.
  DO NOT fix the code until expressly told to do so.

  Your tasks:
  {'- '  + '\n- '.join(steps)}
  '''
  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": f'Code: \n{code}'}
  ]
  response =  client.chat.completions.create(
      model=model,
      messages=messages
  )

  content = response.choices[0].message.content

  return content

### Examples

In [ ]:
def get_examples() -> tuple[list[any], list[str]]:
    '''Returns examples and their labels'''

    # Python examples
    add = r'''
    def add(a, b):
        return a + b
    '''

    multiply = r'''
    def multiply(a, b):
        return a * b
    '''

    divide = r'''
    def divide(a, b):
        return a / b
    '''

    # JavaScript example - async function
    fetch_data = r'''
    async function fetchUserData(userId) {
        const response = await fetch(`/api/users/${userId}`);
        const data = await response.json();
        return data;
    }
    '''

    # Java example - sorting algorithm
    bubble_sort = r'''
    public void bubbleSort(int[] arr) {
        int n = arr.length;
        for (int i = 0; i < n-1; i++) {
            for (int j = 0; j < n-i-1; j++) {
                if (arr[j] > arr[j+1]) {
                    int temp = arr[j];
                    arr[j] = arr[j+1];
                    arr[j+1] = temp;
                }
            }
        }
    }
    '''

    # C++ example - buggy pointer code
    buggy_cpp = r'''
    int* createArray() {
        int arr[5] = {1, 2, 3, 4, 5};
        return arr;
    }
    '''

    # Rust example - ownership puzzle
    rust_ownership = r'''
    fn main() {
        let s1 = String::from("hello");
        let s2 = s1;
        println!("{}", s1);
    }
    '''

    # Go example - concurrent code
    go_concurrent = r'''
    func processData(data []int) int {
        sum := 0
        for _, v := range data {
            sum += v
        }
        return sum
    }
    '''

    # TypeScript example - complex type
    ts_generics = r'''
    function mergeObjects<T, U>(obj1: T, obj2: U): T & U {
        return { ...obj1, ...obj2 };
    }
    '''

    # Ruby example - metaclass magic
    ruby_meta = r'''
    class DynamicMethod
      define_method(:greet) do |name|
        "Hello, #{name}!"
      end
    end
    '''

    # PHP example - SQL injection vulnerable
    php_vulnerable = r'''
    function getUser($id) {
        $query = "SELECT * FROM users WHERE id = " . $id;
        return mysqli_query($conn, $query);
    }
    '''

    # Python example - complex algorithm
    binary_search = r'''
    def binary_search(arr, target):
        left, right = 0, len(arr) - 1
        while left <= right:
            mid = (left + right) // 2
            if arr[mid] == target:
                return mid
            elif arr[mid] < target:
                left = mid + 1
            else:
                right = mid - 1
        return -1
    '''

    # JavaScript example - closure concept
    js_closure = r'''
    function counter() {
        let count = 0;
        return function() {
            count++;
            return count;
        };
    }
    '''

    examples = [
        # Simple Python examples
        [[Task.COMMENTS], add, 'python'],
        [[Task.UNIT_TESTS], multiply, 'python'],
        [[Task.COMMENTS, Task.FIX_CODE], divide, 'python'],

        # Explain complex concepts
        [[Task.EXPLAIN], binary_search, 'python'],
        [[Task.EXPLAIN], js_closure, 'javascript'],
        [[Task.EXPLAIN], rust_ownership, 'rust'],

        # Unit tests for different languages
        [[Task.UNIT_TESTS], fetch_data, 'javascript'],
        [[Task.UNIT_TESTS], go_concurrent, 'go'],

        # Fix buggy code
        [[Task.FIX_CODE], buggy_cpp, 'cpp'],
        [[Task.FIX_CODE], php_vulnerable, 'php'],

        # Multi-task combinations
        [[Task.COMMENTS, Task.EXPLAIN], bubble_sort, None],
        [[Task.COMMENTS, Task.UNIT_TESTS], ts_generics, 'typescript'],
        [[Task.EXPLAIN, Task.FIX_CODE], rust_ownership, 'rust'],
        [[Task.COMMENTS, Task.UNIT_TESTS, Task.EXPLAIN], ruby_meta, 'ruby'],
    ]

    example_labels = [
        '🐍 Python: Add Function',
        '🐍 Python: Multiply Tests',
        '🐍 Python: Fix Division',
        '🐍 Python: Binary Search Explained',
        '🟨 JavaScript: Closure Concept',
        '🦀 Rust: Ownership Puzzle',
        '🟨 JavaScript: Async Test',
        '🐹 Go: Concurrency Test',
        '⚡ C++: Fix Pointer Bug',
        '🐘 PHP: Fix SQL Injection',
        '☕ Java: Bubble Sort Guide',
        '📘 TypeScript: Generics & Tests',
        '🦀 Rust: Fix & Explain Ownership',
        '💎 Ruby: Meta Programming Deep Dive',
    ]

    return examples, example_labels

## Gradio UI

[Documentation](https://www.gradio.app/docs/gradio)

In [ ]:
title = 'Snippet Sniper 🎯'

with gr.Blocks(title=title, theme=gr.themes.Monochrome()) as ui:
    gr.Markdown(f'# {title}')
    gr.Markdown('## I am your [**John Wick**](https://en.wikipedia.org/wiki/John_Wick), ready to accept any contract on your code. Consider it executed 🎯🔫!.')

    with gr.Row():
      with gr.Column():
        tasks = gr.Dropdown(
            label="Tasks",
            choices=[task.value for task in Task],
            value=Task.COMMENTS,
            multiselect=True,
            interactive=True,
        )
        code_input = gr.Code(
            label='Code Input',
            lines=40,
        )
        code_language = gr.Textbox(visible=False)

      with gr.Column():
        gr.Markdown('## Kill Zone 🧟🧠💀')
        code_output = gr.Markdown('💣')


    run_btn = gr.Button('📜 Issue Contract')

    def set_language(tasks, code, language):
      syntax_highlights = ['python',  'c', 'cpp', 'javascript', 'typescript']
      logger.debug(f'Tasks: {tasks}, Languge: {language}')
      highlight = language if language in syntax_highlights else None

      return tasks, gr.Code(value=code, language=highlight)

    examples, example_labels = get_examples()
    examples = gr.Examples(
        examples=examples,
        example_labels=example_labels,
        examples_per_page=20,
        inputs=[tasks, code_input, code_language],
        outputs=[tasks, code_input],
        run_on_click=True,
        fn=set_language
    )

    run_btn.click(perform_tasks, inputs=[tasks, code_input], outputs=[code_output])

ui.launch(debug=True)